# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

.fetchone()
.fetchmany()
.fetchall()
.executemany()
.executescript()

In [1]:
import sqlite3

In [ ]:
con = sqlite3.connect('./data/data/Chinook_Sqlite.sqlite')
cur = con.cursor()


cur.execute("""
SELECT firstname, lastname
FROM Customer
WHERE country = ? AND state = ?
""", ('Canada', 'QC'))

cur.close()
con.close()

In [ ]:
con = sqlite3.connect('./data/data/test.db')
cur = con.cursor()


cur.execute("""
CREATE TABLE Student(id INTEGER, name VARCHAR(50))
""")

cur.execute("""
INSERT INTO Student(id, name) VALUES (1, 'Daniil')
""")
con.commit()

cur.close()
con.close()

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

2. Найти и вывести на экран названия всех альбомов группы Accept

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор.

In [ ]:
conn = sqlite3.connect('./data/data/recipes.db')
curr = conn.cursor()


# curr.close()
# conn.close()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [ ]:
curr.execute("""
CREATE TABLE Recipe(id INTEGER, name VARCHAR(80), minutes TIME)
""")

curr.close()
conn.close()

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

8. Найдите кол-во отзывов с рейтингом 5.

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4.

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.